In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import h5py

import DMsimulator as DMsim
import DMerror as DMerr


In [2]:

#########* Constants and fixed parameters of the model
const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

#########* Steric factors reactions
steric_dict = {
        ###* Atomic oxygen
        "SF_O_F": 1.0, "SF_O_S": 1.0, "SF_O_SO": 1.0, "SF_O_FO": 1.0,
        "SF_FO_S": 1.0, "SF_FO_SO": 1.0, "SF_FO_FO": 1.0, "SF_FO": 1.0,
        
        ###* Molecular oxygen
        "SF_O2_F": 1.0, "SF_O2_FO": 1.0, "SF_O2_FO2": 1.0, "SF_O_FO2": 1.0,
        "SF_FO2_FO": 1.0, "SF_FO_FO2": 1.0, "SF_FO2": 1.0,
        
        ###* Metastable species
        "SF_O2fast_SO": 1.0, "SF_Ofast_SO": 1.0, "SF_O2fast_S": 1.0,  "SF_Ofast_S": 1.0,
        "SF_Ofast_Sdb": 1.0, "SF_Ofast_SOdb": 1.0, "SF_O2fast_Sdb": 1.0, "SF_O2fast_SOdb": 1.0,
        "SF_O_Sdb": 1.0, "SF_O_SOdb": 1.0, "SF_FO_SOdb": 1.0, "SF_FO_Sdb": 1.0,
}

#########* Energy barriers  --  Based Model
energy_dict = { # kJ/mol and s^-1
        "E_O_F": 0.0, "E_O_S": 0.0, "E_O_SO": 15.0, "E_O_FO": 0.0, 
        "E_FO_SO": 20.0, "E_FO_FO":0.0, "E_di_O": 15.0, "E_de_O": 30.0,
        
        "E_O2_F": 0.0, "E_O2_FO": 0.0, "E_O2_FO2": 0.0, "E_O_FO2": 0.0, 
        "E_FO2_FO": 0.0, "E_FO_FO2": 0.0, "E_di_O2": 15.0, "E_de_O2": 17.5,
        
        "E_O2fast_SO": 0.0, "E_O2fast_S": 0.0, "E_O2fast_SOdb": 0.0, "E_O2fast_Sdb": 0.0, "E_Ofast_Sdb": 0.0,
        "E_Ofast_SOdb": 0.0, "E_O_Sdb": 0.0, "E_O_SOdb": 0.0, "E_F_SOdb": 0.0, "E_FO_SOdb": 0.0,
        "ED_db": 14.999,
        
        "nu_D": 1.0e13, "nu_d": 1.0e15,
        
        "Emin": 2.90, # eV
        "Ealpha": 3400.0, # K
        
        # "Emin": 2.60, # eV
        # "Ealpha":  3.23383702e+03, # K
}

file_input_data = "Experimental_data_Paper.hdf5"

In [3]:
with h5py.File('simulations/results_TDPaperWithLowPressureOptimizedCheck.hdf5', 'r') as f:
    
    exp_data = f['exp_data'][:]
    gammas_data = f['gammas_data'][:]
    gammas_names = f['gammas_names'][:].astype(str)
    exp_names = f['exp_names'][:].astype(str)
    recProbExp = f['recProbExp'][:]
    
    f.close()


Tnw_vec = exp_data[:, 0]
Tw_vec = exp_data[:, 1]
pressure_vec = exp_data[:, 3]
current_vec = exp_data[:, 4]
gammas_exp_vec = recProbExp

In [4]:
print("gammas_exp_vec", gammas_exp_vec.shape)
print("gammas_data", gammas_data.shape)

gammas_exp_vec (66,)
gammas_data (66, 12)


In [5]:

class MyErrorModel(DMerr.ErrorPropagation):
    
    def modify_energy_dict(self, counter):
    
        A = 2.74332762e-03
        B = 6.23703570e-04 
        E_nu_d = 1.68496853e+01
        
        Ealpha = 3.26673523e+03 
        Emin = 2.70000000e+00 
        ED_db = 1.00000000e+01
        
        Tw = self.input_data_dict[counter]['Tw']
        R = self.const_dict['R']
        
        
        nu_d = 1e15 * (A + B * np.exp(E_nu_d / (R * Tw)))
        
        self.energy_dict_base['nu_d'] = nu_d
        self.energy_dict_base['Ealpha'] = Ealpha
        self.energy_dict_base['Emin'] = Emin
        self.energy_dict_base['ED_db'] = ED_db
        
        return self.energy_dict_base

In [6]:
system  = MyErrorModel(const_dict, steric_dict, energy_dict, file_input_data)

In [7]:

std_input_ratios_dict = {
    # 'Tnw': 0.02, # lowx
    # 'Tw': 0.01, # low
    'O_den': 0.1,
    'N_den': 0.1
}

# std_input_ratios_dict = {
#     'O_den': 0.1
#     ''
# }


In [8]:


counter = 0


func = lambda x: x
func_squared = lambda x: x**2

value_mean, error_mean = system.compute_mean_func_output(func, std_input_ratios_dict, counter)

print("mean = ", value_mean)
value_squared_mean, error_squared_mean = system.compute_mean_func_output(func_squared, std_input_ratios_dict, counter)

print("mean squared = ", value_squared_mean)

print("value = ", value_mean)
print("error = ", error_mean)

print("std: ", np.sqrt(error_squared_mean - error_mean**2))


print("gammas_exp_vec = ", gammas_exp_vec[counter])
print("gammas_data = ", np.sum(gammas_data[counter]))
print("Tw_vec = ", Tw_vec[counter])
print("pressure_vec = ", pressure_vec[counter])
print("current_vec = ", current_vec[counter])

mean =  0.0006861495426961244
mean squared =  4.2993187058421286e-07
value =  0.0006861495426961244
error =  2.8364358311853076e-15
std:  1.6957685438931956e-05
gammas_exp_vec =  0.0005599904512469901
gammas_data =  0.0006829170414564586
Tw_vec =  253.14999999999998
pressure_vec =  0.4
current_vec =  20.0


In [9]:


counter = 10


func = lambda x: x
func_squared = lambda x: x**2

value_mean, error_mean = system.compute_mean_func_output(func, std_input_ratios_dict, counter)
value_squared_mean, error_squared_mean = system.compute_mean_func_output(func_squared, std_input_ratios_dict, counter)

print("value = ", value_mean)
print("error = ", error_mean)

print("std: ", np.sqrt(error_squared_mean - error_mean**2))


print("gammas_exp_vec = ", gammas_exp_vec[counter])
print("gammas_data = ", np.sum(gammas_data[counter]))
print("Tw_vec = ", Tw_vec[counter])
print("pressure_vec = ", pressure_vec[counter])
print("current_vec = ", current_vec[counter])

value =  0.001052431639330266
error =  5.104823945250214e-15
std:  3.368922858773147e-05
gammas_exp_vec =  0.0011586621617581633
gammas_data =  0.00105251856554718
Tw_vec =  253.14999999999998
pressure_vec =  1.5
current_vec =  40.0


In [10]:

counter = -1


func = lambda x: x
func_squared = lambda x: x**2

value_mean, error_mean = system.compute_mean_func_output(func, std_input_ratios_dict, counter)
value_squared_mean, error_squared_mean = system.compute_mean_func_output(func_squared, std_input_ratios_dict, counter)

print("value = ", value_mean)
print("error = ", error_mean)

print("std: ", np.sqrt(error_squared_mean - error_mean**2))


print("gammas_exp_vec = ", gammas_exp_vec[counter])
print("gammas_data = ", np.sum(gammas_data[counter]))
print("Tw_vec = ", Tw_vec[counter])
print("pressure_vec = ", pressure_vec[counter])
print("current_vec = ", current_vec[counter])

value =  0.0013412716640715193
error =  6.476754421951177e-15
std:  4.086193149425903e-05
gammas_exp_vec =  0.0015278395370210555
gammas_data =  0.0013411094471896044
Tw_vec =  323.15
pressure_vec =  7.5
current_vec =  40.0
